# 自定义量化

参考：[静态量化](https://pytorch.org/tutorials/advanced/static_quantization_tutorial.html)

本教程展示了如何进行训练后的静态量化，并说明了两个更高级的技术——逐通道量化和感知量化的训练——以进一步提高模型的准确性。注意，量化目前 CPU 支持更好，所以在本教程中我们不会使用 GPU/CUDA。在本教程结束时，您将看到 PyTorch 中的量化如何在提高速度的同时显著降低模型大小。此外，您还将看到如何轻松地应用这里所展示的一些高级量化技术，从而使您的量化模型比其他方法获得更少的精度。

In [1]:
# 提供注解的向前兼容
from __future__ import annotations

# 设置 warnings
import warnings
warnings.filterwarnings(
    action='ignore',
    category=DeprecationWarning,
    module=r'.*'
)
warnings.filterwarnings(
    action='default',
    module=r'torch.quantization'
)

from mod import load_mod
load_mod()

本文以 {class}`~torchvision.models.MobileNetV2` 为例，介绍如何将其转换为量化模型 {func}`~torchvision.models.quantization.QuantizableMobileNetV2`。

## 算子融合

```{rubric} 载入库
```

加载一些库：

In [2]:
'''参考 torchvision/models/quantization/mobilenetv2.py
'''
from typing import Any
from torch import Tensor
from torch import nn

from torchvision._internally_replaced_utils import load_state_dict_from_url
from torchvision.ops.misc import ConvNormActivation
from torchvision.models.quantization.utils import _fuse_modules, _replace_relu, quantize_model

from torch.ao.quantization import QuantStub, DeQuantStub
from torchvision.models.mobilenetv2 import InvertedResidual, MobileNetV2, model_urls


__all__ = ["QuantizableMobileNetV2", "mobilenet_v2"]

# 用于微调量化模型
quant_model_urls = {
    "mobilenet_v2_qnnpack": "https://download.pytorch.org/models/quantized/mobilenet_v2_qnnpack_37f702c5.pth"
}

通过几个显著的修改来启用量化的 **算子融合**。

### 用 {class}`torch.nn.quantized.FloatFunctional` 替换加法

In [ ]:
class QuantizableInvertedResidual(InvertedResidual):
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        self.skip_add = nn.quantized.FloatFunctional()

    def forward(self, x: Tensor) -> Tensor:
        if self.use_res_connect:
            return self.skip_add.add(x, self.conv(x))
        else:
            return self.conv(x)

    def fuse_model(self, is_qat: bool | None = None) -> None:
        for idx in range(len(self.conv)):
            if type(self.conv[idx]) is nn.Conv2d:
                _fuse_modules(self.conv,
                              [str(idx),
                               str(idx + 1)],
                              is_qat,
                              inplace=True)